In [1]:
!pip install -U -q transformers==4.39.3
!pip install -U -q accelerate==0.28.0

In [2]:
import os
import torch
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
login(token=user_secrets.get_secret("HUGGINGFACE_TOKEN"))

os.environ["TOKENIZER"]="meta-llama/Llama-2-7b-hf"
os.environ["MODEL"]="apple/OpenELM-270M-Instruct"

# torch.backends.cuda.enable_mem_efficient_sdp(False)
# torch.backends.cuda.enable_flash_sdp(False)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(os.getenv("TOKENIZER"))

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [4]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(os.getenv("MODEL"), trust_remote_code=True)
model.eval()
model.to("cuda")
model.device

config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

configuration_openelm.py:   0%|          | 0.00/14.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/apple/OpenELM-270M-Instruct:
- configuration_openelm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_openelm.py:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/apple/OpenELM-270M-Instruct:
- modeling_openelm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

device(type='cuda', index=0)

In [5]:
query="How the weather today?"

tokenizer_prompt=tokenizer(query, return_tensors='pt').to("cuda")

query_vec=model(**tokenizer_prompt).logits[:,-1,:]
query_vec

tensor([[-13.8810,  -8.5577,   0.5930,  ...,  -8.0688, -10.0788,  -7.0973]],
       device='cuda:0', grad_fn=<SliceBackward0>)

In [6]:
answer="Look. UFO!"

tokenizer_answer=tokenizer(answer, return_tensors='pt').to("cuda")
answer_vec=model(**tokenizer_answer).logits[:,-1,:]
answer_vec

tensor([[ -9.2221, -12.4106,   0.7525,  ...,  -5.1686,  -8.4011,  -5.2176]],
       device='cuda:0', grad_fn=<SliceBackward0>)

In [7]:
answer1="The weather is cold today."

tokenizer_answer1=tokenizer(answer1, return_tensors='pt').to("cuda")
answer_vec1=model(**tokenizer_answer1).logits[:,-1,:]
answer_vec1

tensor([[-14.5745,  -8.6765,  -2.1444,  ...,  -7.4698, -10.4496,  -8.7166]],
       device='cuda:0', grad_fn=<SliceBackward0>)

# Cosine Similarity

The higher the [cosine similarity](https://pytorch.org/docs/stable/generated/torch.nn.CosineSimilarity.html), the closer the semantic meaning of the sentences.

In [8]:
import torch
from termcolor import colored

def compute_cosine_similarity(vec1, vec2):
    cos=torch.nn.CosineSimilarity(dim=1, eps=1e-6)
    return cos(vec1, vec2)

similarity=compute_cosine_similarity(query_vec, answer_vec)

print(colored(f"Query: {query} and answer: {answer}", "green"))
print(colored(f"Cosine Similarity: {similarity}", "blue"))

similarity1=compute_cosine_similarity(query_vec, answer_vec1)

print(colored(f"Query: {query} and answer: {answer1}", "green"))
print(colored(f"Cosine Similarity: {similarity1}", "blue"))

Query: How the weather today? and answer: Look. UFO!
Cosine Similarity: tensor([0.9641], device='cuda:0', grad_fn=<SumBackward1>)
Query: How the weather today? and answer: The weather is cold today.
Cosine Similarity: tensor([0.9927], device='cuda:0', grad_fn=<SumBackward1>)


# Reference

* https://pytorch.org/docs/stable/generated/torch.nn.CosineSimilarity.html
* https://towardsdatascience.com/are-gpts-good-embedding-models-28d8ef6f3f63